# Chapter6 데이터 로딩과 저장, 파일 형식

## 6.1 How to read data from text file

pandas 에서는 표 형식의 자료를 DataFrame 객체로 읽어오는 몇 가지 기능을 제공하고 있다.

주로 read_csv 와 read_table 을 사용한다.

- read_csv : 데이터 구분자는 쉼표로 한다.
- read_table : 데이터 구분자는 탭으로 '\n' 한다.
- read_excel : 엑셀 불러오기
- read_pickle : 파이썬 피클 포맷

이외에도 많으나 대충 저거만 알고 있자

In [2]:
import pandas as pd
import numpy as np

In [2]:
# read_csv로 불러와보자.

df = pd.read_csv(r"C:\Users\thinp\data\ex1.csv") # 자꾸 앞에 r을 써줘야 코드가 먹힌다.

In [3]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
# 구분자 쉽표를 통해 csv 파일도 read_table로 불러올 수 있다.

df2 = pd.read_table(r"C:\Users\thinp\data\ex1.csv", sep=',')

C:\Users\thinp\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
df2

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
# column 이 없는 파일도 있다. header 로 직접 지정해준다.

df3 = pd.read_csv(r"C:\Users\thinp\data\ex2.csv", header=None)
df3

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
# 아니면 column 명을 직접 넣어준다.

df4 = pd.read_csv(r"C:\Users\thinp\data\ex2.csv", names=['a', 'b', 'c', 'd', 'message'])
df4

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
# message 에 대한 column을 색인으로 하는 DataFrame을 반환하려면
# index_col 인자에 4번째 column 또는 'message' 이름을 가진 column을 지정해서 색인으로 만들 수 있다.

names = ['a', 'b', 'c', 'd', 'message']

pd.read_csv(r"C:\Users\thinp\data\ex2.csv",names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


# Chapter7 데이터 정제 및 준비

## 7.1 누락된 데이터 처리하기

In [9]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [10]:
# isnull() 메서드로 결측값을 boolean 객체로 반환 받는다.

string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [11]:
string_data[0] = None

string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

#### NA 처리 메서드

- dropna : 누락된 데이터가 있는 축을 제외 시킨다.
- fillna : 누락된 값을 채우거나, ffill 이나 bfill 같은 보간 메서드를 적용한다.
- isnull : 누락된 값을 불리언 형태로 반환
- notnull : isnull 과 반대되는 메서드

### 7.1.1 누락된 데이터 골라내기

In [12]:
from numpy import nan as NA

data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [13]:
# 위 코드는 다음과 동일하다.

data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [14]:
# dropna 는 기본적으로 NA 값을 하나라도 포함하고 있는 row 를 제외 시킨다.

data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


In [15]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [16]:
# how='all' 옵션을 넘기면 모두 NA 값인 row 만을 제외시킨다.

data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [17]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [18]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [19]:
# DataFrame 의 row 를 제외시키는 방법은 시계열 데이터에 주로 사용되는 경향이 있다.
# 몇 개 이상의 값이 들어 있는 row만 살펴보고 싶다면 thresh 인자에 원하는 값을 넘기면 된다.

df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.859493,NaN,NaN
1,-0.103764,NaN,NaN
2,0.739795,NaN,-1.285624
3,-1.206012,NaN,0.577197
4,1.032727,1.510161,1.206359
5,-2.775225,1.330279,1.145288
6,0.187447,-1.786026,1.142881


In [20]:
df.dropna()

,0,1,2
4,1.032727,1.510161,1.206359
5,-2.775225,1.330279,1.145288
6,0.187447,-1.786026,1.142881


In [21]:
df.dropna(thresh=2)

,0,1,2
2,0.739795,NaN,-1.285624
3,-1.206012,NaN,0.577197
4,1.032727,1.510161,1.206359
5,-2.775225,1.330279,1.145288
6,0.187447,-1.786026,1.142881


### 7.1.2 결측치 채우기

누락된 값을 버리지 않고 구멍을 메우고 싶으면 fillna 메서드를 쓰자.

In [22]:
df.fillna(0)

,0,1,2
0,-0.859493,0.000000,0.000000
1,-0.103764,0.000000,0.000000
2,0.739795,0.000000,-1.285624
3,-1.206012,0.000000,0.577197
4,1.032727,1.510161,1.206359
5,-2.775225,1.330279,1.145288
6,0.187447,-1.786026,1.142881


In [23]:
# fillna 에 dic값을 통해서 column마다 다른 값을 채울 수도 있다.

df.fillna({1:0.5, 2:0})

,0,1,2
0,-0.859493,0.500000,0.000000
1,-0.103764,0.500000,0.000000
2,0.739795,0.500000,-1.285624
3,-1.206012,0.500000,0.577197
4,1.032727,1.510161,1.206359
5,-2.775225,1.330279,1.145288
6,0.187447,-1.786026,1.142881


In [24]:
# fillna 는 새로운 객체를 반환하지만 다음처럼 기존 객체를 변경할 수도 있다.

_ = df.fillna(0, inplace=True)

df

,0,1,2
0,-0.859493,0.000000,0.000000
1,-0.103764,0.000000,0.000000
2,0.739795,0.000000,-1.285624
3,-1.206012,0.000000,0.577197
4,1.032727,1.510161,1.206359
5,-2.775225,1.330279,1.145288
6,0.187447,-1.786026,1.142881


In [25]:
# 재색인에서 사용가능한 보간 메서드는 fillna 메서드에서도 사용 가능하다.

df = pd.DataFrame(np.random.randn(6,3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA

df

,0,1,2
0,1.532065,-0.178192,-0.010345
1,-1.008473,-0.495334,1.037181
2,2.028009,NaN,0.894159
3,0.199668,NaN,-0.256579
4,-1.641856,NaN,NaN
5,1.187682,NaN,NaN


In [26]:
df.fillna(method='ffill') # 앞 색인값으로 채우겠다.

,0,1,2
0,1.532065,-0.178192,-0.010345
1,-1.008473,-0.495334,1.037181
2,2.028009,-0.495334,0.894159
3,0.199668,-0.495334,-0.256579
4,-1.641856,-0.495334,-0.256579
5,1.187682,-0.495334,-0.256579


In [27]:
df.fillna(method='ffill', limit=2) # 근데 두개까지만 채워주라.

,0,1,2
0,1.532065,-0.178192,-0.010345
1,-1.008473,-0.495334,1.037181
2,2.028009,-0.495334,0.894159
3,0.199668,-0.495334,-0.256579
4,-1.641856,NaN,-0.256579
5,1.187682,NaN,-0.256579


In [28]:
# fillna 를 이용해서 창의적으로 채울 수 있다.
# mean() 이나 median() 등을 활용하는 것.

data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

#### fillna 함수 인자

- value : 비어있는 값을 채울 스칼라값이나 사전 형식의 객체
- method : 보간 방식. 기본적으로 'ffill'을 사용한다.
- axis : 축. 기본은 =0
- inplace : 복사본 생성않고 호출한 객체 변경. 기본값은 False
- limit : 몇 개까지 채울지

## 7.2 데이터 변형

### 7.2.1 중복 제거하기

In [29]:
data = pd.DataFrame({'k1':['one', 'two']*3 + ['two'],
                    'k2':[1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [30]:
# duplicated 메서드는 각 row 가 중복인지 아닌지 알려주는 boolean Series를 반환한다.

data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [31]:
# drop_duplicates() 는 중복되지 않은 값을 반환한다.

data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [32]:
# 부분합을 따로 지정해줄 수도 있다.

data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [33]:
# keep='last' 옵션을 넘기면 마지막으로 발견된 값을 반환한다.

data.drop_duplicates(['k1', 'k2'], keep='last')

# 5번 row가 사라지고 6번 로우가 반환되는 것을 볼 수 있다.

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 7.2.2 함수나 매핑을 이용해 데이터 변경하기

In [34]:
# DataFrame 의 column이나 Series, array 내의 값을 기반으로 데이터의 형태를 변환하고 싶은 경우가 있다.

data = pd.DataFrame({'food':['bacon', 'pulled pork', 'bacon', 'pastrami', 'corned beef', 'Bacon', 'pastrami',
                            'honey ham', 'nova lox'],
                    'ounces':[4,3,12,6,7.5,8,3,5,6]})

data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [38]:
# 육류별 동물 사전 데이터를 작성하자.

meat_to_animal = {'bacon':'pig',
                  'pulled pork':'pig',
                 'pastrami':'cow',
                 'corned beef':'cow',
                 'honey ham':'pig',
                 'nova lox':'salmon'}

In [39]:
# 이름에 대소문자가 섞여 있으므로 str.lower 메서드를 통해 변경해준다.

lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [40]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [41]:
# lambda 함수로 같은 일을 수행 할 수도 있다.
# map 메서드를 사용하면 데이터의 요소별 변환 및 데이터를 다듬는 작업을 편리하게 수행할 수 있다.

data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 7.2.3 값 치환하기

In [42]:
# fillna 메서드를 사용해서 누락된 값을 채우는 일은 일반적인 값 치환 작업이라고 볼 수 있다.
# map 메서드를 한 객체 안에서 값의 부분집합을 변경하는 데 사용했다면
# replace 메서드는 같은 작업에 좀 더 간단하고 유연한 방법을 제공한다.

data = pd.Series([1, -999, 2., -999., -1000, 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [43]:
# -999는 누락된 데이터를 나타내기 위한 값이다.
# replace 메서드를 이용하면 이 값을 pandas 에서 인식할 수 있는 NA값으로 치환한 새로운 Series를 생성한다.

data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [44]:
# 여러 개를 한 번에 치환할 수도 있다.

data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [45]:
# 치환하려는 값마다 다르게 하려면 리스트를 사용하자.

data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [47]:
# 두 개의 리스트 대신 딕셔너리를 이용하는 것도 가능

data.replace({-999:np.nan, -1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 7.2.4 축 색인 이름 바꾸기

In [56]:


data = pd.DataFrame(np.arange(12).reshape((3,4)),
                   index = ['seoul', 'tokyo', 'taibei'],
                   columns= ['one', 'two', 'three', 'four'])

In [57]:
transform = lambda x: x[:].upper()

In [58]:
data.index.map(transform)

Index(['SEOUL', 'TOKYO', 'TAIBEI'], dtype='object')

In [59]:
대문자로 변경된 축 이름을 DataFrame 의 index에 바로 대입.

data.index = data.index.map(transform)
data

,one,two,three,four
SEOUL,0,1,2,3
TOKYO,4,5,6,7
TAIBEI,8,9,10,11


In [60]:
# 원래 객체를 변경하지 않고 새로운 객채를 생성하려면 rename 메서드를 사용한다.

data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Seoul,0,1,2,3
Tokyo,4,5,6,7
Taibei,8,9,10,11


In [62]:
# 특히 rename 메서드는 사전 형식의 객체를 이용해서 축 이름 중 일부만 변경하는 것도 가능

data.rename(index={'SEOUL':'PUSAN'}, columns = {'three':'peekaboo'})

,one,two,peekaboo,four
PUSAN,0,1,2,3
TOKYO,4,5,6,7
TAIBEI,8,9,10,11


In [65]:
# rename 메서드를 사용하면 DataFrame을 직접 복사해서 index 와 columns 속성을 갱신할 필요없이 바로 변경할 수 있다.
# 원본 데이터를 바로 변경하려면 inplace=True 옵션을 넘겨주면 된다.

data.rename(index={'SEOUL':'PUSAN'}, inplace=True)
data

,one,two,three,four
PUSAN,0,1,2,3
TOKYO,4,5,6,7
TAIBEI,8,9,10,11


### 7.2.5 개별화와 양자화

연속성 데이터는 종종 개별로 분할하거나 아니면 분석을 위해 그룹별로 나누기도 한다.

In [66]:
ages = [20,22,25,27,21,23,37,31,61,45,41,32]

In [67]:
bins = [18,25,35,60,100]

In [68]:
cats = pd.cut(ages, bins)

In [69]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [70]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [71]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [72]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [73]:
# 간격을 나타내는 표기법은 중괄호로 시작해서 대괄호로 끝나는데
# 중괄호 쪽의 값은 포함하지 않고 대괄호 쪽의 값은 포함
# right=False 를 넘겨서 중괄호 대신 대괄호쪽이 포함되지 않도록 바꿀 수 있다.

pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [74]:
# labels 옵션으로 그룹의 이름을 직접 넘겨줄 수도 있다.

group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels = group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [79]:
# cut 함수에 명시적으로 그룹의 경계값을 넘기지 않고 그룹의 개수를 넘겨주면
# 데이터에서 최솟값과 최댓값을 기준으로 균등한 길이의 그룹을 자동으로 계산한다.

data = np.random.rand(20) # rand와 randn 의 차이가 궁금할 것이다. rand는 가우시안분포, randn은 표준정규분포에서 난수 추출.
pd.cut(data, 4, precision = 2)

# precision=2 옵션은 소수점 아래 2자리까지로 제한한다.

[(0.24, 0.47], (0.71, 0.95], (0.24, 0.47], (0.24, 0.47], (-0.00044, 0.24], ..., (0.24, 0.47], (-0.00044, 0.24], (-0.00044, 0.24], (0.71, 0.95], (-0.00044, 0.24]]
Length: 20
Categories (4, interval[float64]): [(-0.00044, 0.24] < (0.24, 0.47] < (0.47, 0.71] < (0.71, 0.95]]

In [80]:
# 이를 위한 가장 적합한 함수로 qcut 이 있는데, 표본 변위치를 기반으로 데이터를 나눠준다.
# cut은 데이터의 분산에 따라 가각의 그룹마다 데이터 수가 다르게 나뉘는 경우가 많다.
# qcut은 표준 변위치를 사용하기 때문에 적당하게 같은 크기로 나눈다.

data = np.random.randn(1000)
cats = pd.qcut(data,4)
cats

[(-0.63, 0.0253], (0.651, 3.487], (-0.63, 0.0253], (0.0253, 0.651], (0.651, 3.487], ..., (0.651, 3.487], (-3.127, -0.63], (-0.63, 0.0253], (0.651, 3.487], (0.0253, 0.651]]
Length: 1000
Categories (4, interval[float64]): [(-3.127, -0.63] < (-0.63, 0.0253] < (0.0253, 0.651] < (0.651, 3.487]]

In [81]:
pd.value_counts(cats)

(0.651, 3.487]     250
(0.0253, 0.651]    250
(-0.63, 0.0253]    250
(-3.127, -0.63]    250
dtype: int64

In [82]:
# cut 함수와 유사하게 변위치를 직접 지정해줄 수 있다.(변위치는 0부터 1까지)

pd.qcut(data, [0, 0.1, 0.5, 0.9, 1])

[(-1.248, 0.0253], (1.197, 3.487], (-1.248, 0.0253], (0.0253, 1.197], (0.0253, 1.197], ..., (1.197, 3.487], (-1.248, 0.0253], (-1.248, 0.0253], (0.0253, 1.197], (0.0253, 1.197]]
Length: 1000
Categories (4, interval[float64]): [(-3.127, -1.248] < (-1.248, 0.0253] < (0.0253, 1.197] < (1.197, 3.487]]

### 7.2.6 특잇값을 찾고 제외하기

In [83]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.026504,0.005276,-0.033561,-0.019485
std,1.022320,0.998426,1.013940,0.983914
min,-3.487374,-3.317225,-3.050138,-2.946467
25%,-0.638473,-0.658344,-0.761641,-0.706232
50%,0.044202,-0.017047,-0.051792,-0.004219
75%,0.709872,0.687627,0.638597,0.687134
max,3.057217,3.678834,3.527087,3.340886


In [84]:
# 이 DF 에서 한 column에서 절댓값이 3을 초과하는 값을 찾아내자.

col = data[2]
col[np.abs(col) > 3]

89     3.216517
343    3.103339
605    3.527087
736   -3.050138
Name: 2, dtype: float64

In [85]:
# 절댓값이 3을 초과하는 값이 들어 있는 모든 row를 선택하려면 불리언 DataFrame에서 any 메서드를 사용하면 된다.

data[(np.abs(data) > 3).any(1)]

,0,1,2,3
27,-3.010731,0.570359,1.259744,0.468943
89,-0.009925,1.587365,3.216517,-1.051346
201,-0.250770,-3.317225,0.662621,0.063872
280,-0.332306,2.186858,-0.815529,3.340886
330,3.057217,-1.374796,-0.591971,0.236134
343,1.090134,-0.183228,3.103339,-1.941315
570,-0.277161,3.255698,0.907505,0.153864
605,1.225686,1.166263,3.527087,0.359971
692,0.792214,3.678834,-0.495612,-0.905828
736,-0.273331,-0.140721,-3.050138,1.122622


In [86]:
# 아래 코드로 -3이나 3을 초과하는 값을 -3 또는 3으로 지정할 수 있다.

data[np.abs(data) > 3] = np.sign(data) *3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.027005,0.004659,-0.034358,-0.019826
std,1.020376,0.994341,1.011085,0.982807
min,-3.000000,-3.000000,-3.000000,-2.946467
25%,-0.638473,-0.658344,-0.761641,-0.706232
50%,0.044202,-0.017047,-0.051792,-0.004219
75%,0.709872,0.687627,0.638597,0.687134
max,3.000000,3.000000,3.000000,3.000000


In [87]:
# np.sign(data) 는 data 값이 양수인지 음수인지에 따라 1이나 -1이 담긴 배열을 반환한다.

np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,1.0
1,1.0,1.0,-1.0,1.0
2,-1.0,-1.0,1.0,-1.0
3,1.0,1.0,-1.0,1.0
4,1.0,-1.0,-1.0,1.0


### 7.2.7 치환과 임의 샘플링

In [3]:
# numpy.random.permutation 함수를 이용하면 Series나 DataFrame의 row를 쉽게 임의로 재배치 할 수 있다.
# 순서를 바꾸고 싶은 만큼의 길이를 permutation 함수로 넘기면 된다.

df = pd.DataFrame(np.arange(5 *4).reshape((5,4)))
sampler = np.random.permutation(5)
sampler

array([3, 1, 0, 4, 2])

In [4]:
# 이 배열은 iloc 기반의 색인이나 take 함수에서 사용 가능하다.

df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [5]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
1,4,5,6,7
0,0,1,2,3
4,16,17,18,19
2,8,9,10,11


In [6]:
# 치환없이 일부로 임의로 선택하려면 Series나 DataFrame의 sample 메서드를 사용하면 된다.

df.sample(n=3)

,0,1,2,3
1,4,5,6,7
2,8,9,10,11
0,0,1,2,3


In [7]:
# 표본을 치환을 통해 생성해내려면 sample 에 replace=True 옵션을 넘긴다.

choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

2   -1
2   -1
4    4
4    4
0    5
2   -1
0    5
0    5
0    5
3    6
dtype: int64

### 7.2.8 표시자/더미 변수 계산하기

통계 모델이나 머신러닝 어플을 위한 또 다른 데이터 변환은 분류값을 '더미'나 '표시자' 행렬로 전환하는 것이다.

In [8]:
df = pd.DataFrame({'key':['b', 'b', 'a', 'c', 'a', 'b'],
                  'data1':range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [10]:
# column에 접두어를 추가한 후 다른 데이터와 병합하고 싶을 경우가 있다.
# get_dummies 함수에 prefix 인자를 사용하자.

dummies = pd.get_dummies(df['key'], prefix='key') # 기존의 column 명에 key를 접두어로 넣는다.
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


## 7.3 문자열 다루기

### 7.3.1 문자열 객체 메서드

In [11]:
# 문자열을 다루는 대부분의 애플리케이션은 내장 함수만으로도 충분함
# 쉼표로 구분된 문자열은 split 메서드를 이용하자.

val = 'a, b, guido'

val.split(',')

['a', ' b', ' guido']

In [12]:
# split 메서드는 종종 공백 문자를 제거하는 strip 메서드와 조합해서 사용하기도 한다.

pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [13]:
# 이렇게 분리된 문자열은 더하기 연산을 사용해서 :: 문자열과 합칠 수도 있다.

first, second, third = pieces

first + '::' + second + '::' + third

'a::b::guido'

In [14]:
'::'.join(pieces)

'a::b::guido'

In [16]:
# 일치하는 부분문자열의 위치를 찾는 방법도 있다. index나 find 를 사용하는 것도 가능하지만 파이썬의 in 예약어를 사용하면 쉽다.

'guido' in val

True

In [17]:
val.index(',')

1

In [18]:
val.find(':')

-1

In [19]:
# find 와 index 의 차이점은 index의 경우 문자열을 찾지 못하면 예외를 발생시킨다는 것이다.
# find의 경우에는 -1 을 반환한다.

val.index(':')

ValueError: substring not found

In [20]:
# count는 특정 부분 문자열이 몇 건 발견되었는지 반환한다.

val.count(', ')

2

In [21]:
# replace 는 찾아낸 패턴을 다른 문자열로 치환한다.
# 이 메서드는 대체할 문자열로 비어 있는 문자열을 넘겨서 패턴을 삭제하기 위한 방법으로 자주 사용되기도 한다.

val.replace(',', '::')

'a:: b:: guido'

In [22]:
val.replace(',', '')

'a b guido'

#### 파이썬 내장 문자열 함수

- count
- endswith : 문자열이 주어진 접미사로 끝날 경우 True 를 반환
- startwith : 반대다.
- join : 문자열을 구분자로 하여 다른 문자열을 순서대로 이어붙인다.
- index : 부분문자열의 첫 번째 글자의 위치를 반환
- find : 같다.
- rfind : 마지막 부분문자열의 첫 번째 글자의 위치를 반환
- replace: 치환
- strip, rstrip, lstrip : 개행 문자를 포함한 공백문자를 제거. r은 마지막 부분, l은 시작 부분.
- split : 구분자를 기준으로 부분문자열의 리스트로 분리
- lower : 소문자로
- upper : 대문자로
- caseford : 문자를 소문자로 변환, 지역문자들은 그에 상응하는 대체 문자로 교체
- ljust, rjust : 문자열을 오른쪽 또는 왼쪽으로 정렬하고 주어진 길이에서 문자열의 길이를 제외한 나머지 부분은 공백 문자를 채워 넣는다.

### 7.3.2 정규 표현식

파이썬에는 re 모듈 함수가 있다.

In [23]:
# re 모듈 함수는 패턴 매칭, 치환, 분리 세 가지로 나눌 수 있다.
# 여러개의 공백문자(탭, 스페이스, 개행문자) 가 포함된 문자열을 나누고 싶다면 하나 이상의 공백 문자를 의미하는 \s+ 를 사용

import re

In [24]:
text = "foo bar\t baz \tqux"

In [25]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

In [26]:
# re.split 을 사용하면 먼저 정규 표현식이 컴파일되고 그 다음에 split 메서드가 실행된다.
# re.compile 로 직접 정규 표현식을 컴파일하고 그렇게 얻은 정규 표현식 객체를 재사용하는 것도 가능하다.

regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [27]:
# 정규 표현식에 매칭되는 모든 패턴의 목록을 얻고 싶다면 findall 메서드를 사용하낟.

regex.findall(text)

[' ', '\t ', ' \t']

In [29]:
# match와 search 는 findall 메서드와 관련이 있다.
# findall 은 문자열에서 일치하는 모든 부분 문자열을 찾아주지만
# search 메서드는 패턴과 일치하는 첫 번째 존재를 반환한다.
# match 메서드는 이보다 더 엄격해서 문자열의 시작부분에서 일치하는 것만 찾아준다.

text = """Dave dave@gmail.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com"""

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE는 정규 표현식이 대소문자를 가리지 않도록 한다.
regex = re.compile(pattern, flags = re.IGNORECASE)

In [31]:
# findall 메서드를 사용해서 이메일 주소의 리스트를 생성하자.

regex.findall(text)

['dave@gmail.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [33]:
# search 랑 match를 써보자

m = regex.search(text)
m

<re.Match object; span=(5, 19), match='dave@gmail.com'>

In [35]:
text[m.start():m.end()]

'dave@gmail.com'

In [36]:
# regex.match 는 None을 반환한다. 왜냐하면 그 정규 표현 패턴이 문자열의 시작점에서부터 일치하는지 검사하기 때문이다.

print(regex.match(text))

None


In [37]:
# sub 메서드는 찾은 패턴을 주어진 문자열로 치환하여 새로운 문자열을 반환한다.

print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED


In [38]:
# 이메일 주소를 찾아서 동시에 각 이메일 주소를 사용자 이름, 도메인 이름, 도메인 접미사 세가지 컴포넌트로 나눈다면
# 괄호로 묶어 준다.

pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [39]:
m = regex.match('wesm@brignt.net')
m.groups()

('wesm', 'brignt', 'net')

In [40]:
# 패턴에 그룹이 존재한다면 findall 메서드는 튜플의 목록을 반환한다.

regex.findall(text)

[('dave', 'gmail', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [41]:
# sub 역시 마찬가지로 \1, \2 같은 특수한 기호를 사용해서 각 패턴 그룹에 접근할 수 있다.
# \1 은 첫 번째로 찾은 그룹, \2는 두 번째로 찾은 그룹을 의미

print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: gmail, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com


### 7.3.3 pandas 의 벡터화된 문자열 함수

In [42]:
# 누락된 값이 있다.

data = {'Dave': 'dave@gmail.com','Steve': 'steve@gmail.com',
        'Rob' : 'rob@gmail.com',
        'Wes' : np.nan}

data = pd.Series(data)
data

Dave      dave@gmail.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [43]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [44]:
# 문자열과 정규 표현식 메서드는 data.map 을 사용해서 각 값에 적용할 수 있지만 NA값을 만나면 실패하게 된다.
# Series에서는 str 속성을 이용해서 str.contains 를 이용한다.

data.str.contains('gmail')

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [45]:
# 정규 표현식을 IGNORECASE 같은 re 옵션과 함께 사용하는 것도 가능하다.

pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [46]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave      [(dave, gmail, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [48]:
# 벡터화된 요소를 꺼내오는 몇 가지 방법이 있는데 str.get 을 이용하거나 str 속성의 색인을 이용한다.

matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [49]:
# 내재된 리스트의 원소에 접근하기 위해서는 색인을 넘기면 된다.

matches.str.get(1)

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [50]:
matches.str[0]

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [51]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object